# Product Prompt Assistant
AI Marketing Content Generator

Generate Creative Marketing Content Instantly

In [1]:
%pip install flask flask_sqlalchemy python-dotenv nest_asyncio requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import json
import requests
from datetime import datetime
from flask import Flask, request, jsonify, send_from_directory
from flask_sqlalchemy import SQLAlchemy
from dotenv import load_dotenv
import nest_asyncio
from threading import Thread
from pathlib import Path

# Allow Flask to run inside Jupyter
nest_asyncio.apply()

# Load API key
load_dotenv()
API_KEY = os.getenv("GEMINI_API_KEY")

if not API_KEY:
    raise ValueError("⚠️ GEMINI_API_KEY not found in .env file")

print("✅ Gemini API Key loaded successfully")

✅ Gemini API Key loaded successfully


In [3]:
# Cell 3: MarketingContentAssistant.ipynb

# Load prompt templates from the JSON file
TEMPLATES_FILE = Path.cwd() / "templates" / "prompt_templates.json"
try:
    with open(TEMPLATES_FILE, 'r') as f:
        PROMPT_TEMPLATES = json.load(f)
    print("✅ Prompt templates loaded successfully")
except FileNotFoundError:
    raise FileNotFoundError(f"Template file not found at: {TEMPLATES_FILE}")


GEMINI_URL = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent"

def generate_marketing_content(product_name, description, audience="general audience",
                               tone="friendly", template="ad_copy", n_variants=2):
    """Generate marketing content using Gemini API by loading templates."""
    
    # --- 🌟 ADVANCED FEATURE 1: Template Loading Logic ---
    template_key = template.replace('_', '') # Normalize key: e.g., 'ad_copy' -> 'adcopy'
    
    # Check if a specific template exists, otherwise fall back to a generic prompt
    if template_key in PROMPT_TEMPLATES:
        raw_prompt = PROMPT_TEMPLATES[template_key]
        
        # Replace placeholders in the template
        prompt = raw_prompt.format(
            n_variants=n_variants,
            product_name=product_name,
            # Map the detailed 'description' field to the template's '{product_value}'
            product_value=description, 
            audience=audience,
            tone=tone
        )
    else:
        # Fallback to the generic prompt if template key is missing (Good for robustness)
        print(f"⚠️ Template '{template}' not found. Using generic prompt.")
        prompt = f"""
        Generate {n_variants} {template.replace('_', ' ')} variants for a product named '{product_name}'.
        Description: {description}.
        Audience: {audience}.
        Tone: {tone}.
        Make each variant engaging and visually distinct (use line breaks).
        """
    # --- END ADVANCED FEATURE 1 ---

    headers = {"Content-Type": "application/json"}
    payload = {"contents": [{"role": "user", "parts": [{"text": prompt}]}]}

    response = requests.post(f"{GEMINI_URL}?key={API_KEY}", headers=headers, json=payload)
    if response.status_code != 200:
        return f"❌ API Error: {response.status_code} - {response.text}"

    try:
        data = response.json()
        text = data["candidates"][0]["content"]["parts"][0]["text"]
        return text.strip()
    except Exception as e:
        print("⚠️ Parsing error:", e)
        return "⚠️ Unexpected response format from Gemini API"

✅ Prompt templates loaded successfully


In [4]:
# Cell 4: DEFINITIVE FIX for SQLAlchemy Registration Error

import os
from pathlib import Path
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime
# Note: Ensure you have imported 'db' and 'app' in Cell 5 from this notebook scope.

# Path setup (UNCHANGED)
BASE_DIR = Path.cwd() 
FRONTEND_DIR = BASE_DIR / "frontend"
INSTANCE_DIR = BASE_DIR / "instance"
os.makedirs(INSTANCE_DIR, exist_ok=True)


# --- 🌟 FIX: Reset db if it exists to clear previous notebook state 🌟 ---
# We use a simple check to see if 'db' is already defined in the global scope 
# and explicitly set it to None, then rely on the robust init_app pattern.
try:
    if 'db' in globals():
        global db
        db = None 
except NameError:
    pass 
# --- END FIX ---


# 1. Initialize Flask App
app = Flask(__name__, 
            static_folder=str(FRONTEND_DIR), 
            static_url_path="/"
           )

# 2. Initialize SQLAlchemy WITHOUT an app (Deferred Initialization)
db = SQLAlchemy()


# 3. Configure App
app.config["SQLALCHEMY_DATABASE_URI"] = f"sqlite:///{os.path.join(INSTANCE_DIR, 'productprompt.db')}"
app.config["SQLALCHEMY_TRACK_MODIFICATIONS"] = False


# 4. Bind SQLAlchemy to App using init_app()
db.init_app(app)


class GeneratedContent(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    product_name = db.Column(db.String(200), nullable=False)
    tone = db.Column(db.String(100))
    template = db.Column(db.String(100))
    generated_text = db.Column(db.Text, nullable=False)
    created_at = db.Column(db.DateTime, default=datetime.utcnow)
    favorite = db.Column(db.Boolean, default=False)

# 5. Create Tables within the Application Context
with app.app_context():
    db.create_all()

print("✅ Database initialized successfully.")

✅ Database initialized successfully.


In [5]:
# Cell 5: Flask Routes with Session Cleanup

@app.teardown_appcontext
def shutdown_session(exception=None):
    """Ensures the database session is removed after each request."""
    db.session.remove()

@app.route("/")
def index():
    return send_from_directory(app.static_folder, "index.html")

@app.route("/<path:path>")
def static_files(path):
    return send_from_directory(app.static_folder, path)

@app.route("/generate", methods=["POST"])
def generate():
    try:
        data = request.get_json()
        product_name = data.get("product_name")
        description = data.get("description")
        audience = data.get("audience", "general audience")
        tone = data.get("tone", "friendly")
        template = data.get("template", "ad_copy")
        n_variants = int(data.get("n_variants", 2))

        if not product_name or not description:
            return jsonify({"status": "error", "message": "Product name and description required"}), 400

        output_text = generate_marketing_content(product_name, description, audience, tone, template, n_variants)

        # Using app.app_context() here is defensive but often not needed when using db.init_app(app)
        # However, it ensures the commit happens correctly in the notebook environment.
        with app.app_context():
            entry = GeneratedContent(
                product_name=product_name,
                tone=tone,
                template=template,
                generated_text=output_text
            )
            db.session.add(entry)
            db.session.commit()

        return jsonify({
            "status": "success",
            "product_name": product_name,
            "text": output_text
        })
    except Exception as e:
        return jsonify({"status": "error", "message": str(e)})


@app.route("/history/<int:item_id>", methods=["DELETE"])
def delete_history(item_id):
    """Deletes a history item by ID."""
    try:
        entry = db.session.get(GeneratedContent, item_id)
        if entry is None:
            return jsonify({"status": "error", "message": "Item not found"}), 404
        
        db.session.delete(entry)
        db.session.commit()
        return jsonify({"status": "success", "message": "Item deleted"})
    except Exception as e:
        return jsonify({"status": "error", "message": str(e)}), 500


@app.route("/history/favorite/<int:item_id>", methods=["POST"])
def toggle_favorite(item_id):
    """Toggles the favorite status of a history item by ID."""
    try:
        entry = db.session.get(GeneratedContent, item_id)
        if entry is None:
            return jsonify({"status": "error", "message": "Item not found"}), 404
        
        entry.favorite = not entry.favorite  # Toggle the boolean status
        db.session.commit()
        return jsonify({"status": "success", "favorite": entry.favorite})
    except Exception as e:
        return jsonify({"status": "error", "message": str(e)}), 500

@app.route("/history", methods=["GET"])
def history():
    try:
        entries = GeneratedContent.query.order_by(
            GeneratedContent.favorite.desc(), 
            GeneratedContent.created_at.desc()
        ).limit(10).all()
        
        return jsonify([{
            "id": e.id, 
            "product_name": e.product_name,
            "tone": e.tone,
            "template": e.template,
            "generated_text": e.generated_text,
            "created_at": e.created_at.strftime("%Y-%m-%d %H:%M:%S"),
            "favorite": e.favorite 
        } for e in entries])
    except Exception as e:
        return jsonify({"status": "error", "message": str(e)}), 500

In [ ]:
def run_flask():
    app.run(port=5000, debug=False, use_reloader=False)

print("🚀 ProductPrompt running at: http://127.0.0.1:5000")
thread = Thread(target=run_flask)
thread.start()

🚀 ProductPrompt running at: http://127.0.0.1:5000


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [02/Dec/2025 15:19:13] "POST /generate HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2025 15:24:22] "GET / HTTP/1.1" 304 -
127.0.0.1 - - [02/Dec/2025 15:24:22] "GET /script.js HTTP/1.1" 304 -
127.0.0.1 - - [02/Dec/2025 15:24:23] "GET /style.css HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2025 15:24:26] "GET /history HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2025 15:24:30] "GET / HTTP/1.1" 304 -
127.0.0.1 - - [02/Dec/2025 15:24:30] "GET /style.css HTTP/1.1" 304 -
127.0.0.1 - - [02/Dec/2025 15:24:30] "GET /script.js HTTP/1.1" 304 -
127.0.0.1 - - [02/Dec/2025 15:24:30] "GET /history HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2025 15:28:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2025 15:28:08] "GET /script.js HTTP/1.1" 304 -
127.0.0.1 - - [02/Dec/2025 15:28:08] "GET /style.css HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2025 15:28:08] "GET /history HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2025 16:55:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/D